In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_breast_cancer, fetch_california_housing, load_diabetes
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from scipy.stats import spearmanr, pearsonr

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
# path_sp  = "C:/Users/46705/Documents/SpiderSilk/generalized/filtered_spidroin.csv"
path_anno = "C:/Users/46705/Documents/SpiderSilk/data/pre_filtering/spidroins_cpm_annotated.csv"


df_anno = pd.read_csv(path_anno, sep= ",")
path_info  = "C:/Users/46705/Documents/SpiderSilk/data/annotations/Lsc.v1.1.genes.AA.signalP.lascID.info.csv"

df_info = pd.read_csv(path_info, sep=',')
path_extra_anno = "C:/Users/46705/Downloads/glandSpecific.clusterZones.2 (1).csv"
csv_add_anno = pd.read_csv(path_extra_anno, sep=",")


In [5]:
combined_anno = pd.merge(csv_add_anno, df_info, on="geneName", how= "right")
combined_anno

In [7]:
combined_anno[combined_anno["lascID"]== "LASC019421"]


,geneName,cluster,avg_log2FC,p_val_adj,pct.1,pct.2,rank,geneNameProtein,silkGroup,lascID,product,Group,aa_len,tx_len,gene_id,tx_name
1963,LASC019421,ZoneC_SpiCE-LMa1,8.801834,5.245092e-146,0.971,0.104,19.0,NaN,NaN,LASC019421,hypothetical protein,NaN,95,285,augustus_masked-000381F-processed-gene-6.12,augustus_masked-000381F-processed-gene-6.12-mR...


In [2]:

orthoG = "C:/Users/46705/Documents/SpiderSilk/data/annotations/Orthogroups_ls.tsv"
df_orthogroups = pd.read_csv(orthoG, sep='\t') 
first_last_columns = df_orthogroups.iloc[:, [0, -1]]
first_last_columns

C:\Users\46705\AppData\Local\Temp\ipykernel_3592\2947152215.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255

MemoryError: Unable to allocate 371. MiB for an array with shape (293, 165910) and data type object

In [9]:

first_last_columns['Lsc.v1.1.genes.AA.signalP.lascID'] = first_last_columns['Lsc.v1.1.genes.AA.signalP.lascID'].str.split(', ')
first_last_columns_exploded = first_last_columns.explode('Lsc.v1.1.genes.AA.signalP.lascID')


C:\Users\46705\AppData\Local\Temp\ipykernel_18684\1001108444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_last_columns['Lsc.v1.1.genes.AA.signalP.lascID'] = first_last_columns['Lsc.v1.1.genes.AA.signalP.lascID'].str.split(', ')


In [10]:
first_last_columns_exploded

,Orthogroup,Lsc.v1.1.genes.AA.signalP.lascID
0,OG0000000,LASC000575
0,OG0000000,LASC000643
0,OG0000000,LASC000881
0,OG0000000,LASC001039
0,OG0000000,LASC001378
...,...,...
165907,OG0165907,LASC022621
165908,OG0165908,LASC022514
165908,OG0165908,LASC022591
165909,OG0165909,LASC022526


In [11]:

merged_combined = pd.merge(combined_anno, first_last_columns_exploded[['Lsc.v1.1.genes.AA.signalP.lascID', 'Orthogroup']], left_on='lascID', right_on='Lsc.v1.1.genes.AA.signalP.lascID', how='left')
combined_anno

,geneName,cluster,avg_log2FC,p_val_adj,pct.1,pct.2,rank,geneNameProtein,silkGroup,lascID,product,Group,aa_len,tx_len,gene_id,tx_name
0,LASC000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LASC000004,hypothetical protein,NaN,98,294,augustus_masked-000000F-processed-gene-0.6,augustus_masked-000000F-processed-gene-0.6-mRNA-1
1,LASC000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LASC000006,hypothetical protein,NaN,67,201,augustus_masked-000000F-processed-gene-0.7,augustus_masked-000000F-processed-gene-0.7-mRNA-1
2,dnttip1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LASC000219,Deoxynucleotidyltransferase terminal-interacti...,NaN,492,1476,augustus_masked-000000F-processed-gene-104.5,augustus_masked-000000F-processed-gene-104.5-m...
3,LASC000215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LASC000215,hypothetical protein,NaN,217,651,augustus_masked-000000F-processed-gene-104.6,augustus_masked-000000F-processed-gene-104.6-m...
4,rdx_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LASC000229,Protein roadkill,NaN,345,1035,augustus_masked-000000F-processed-gene-107.3,augustus_masked-000000F-processed-gene-107.3-m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23191,LASC022848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LASC022848,hypothetical protein,NaN,41,291,maker-001775F-est_gff_StringTie-gene-0.0,maker-001775F-est_gff_StringTie-gene-0.0-mRNA-1
23192,LASC022849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LASC022849,hypothetical protein,NaN,137,748,maker-001783F-est_gff_StringTie-gene-0.0,maker-001783F-est_gff_StringTie-gene-0.0-mRNA-1
23193,LASC022851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LASC022851,hypothetical protein,NaN,40,279,maker-001812F-est_gff_StringTie-gene-0.0,maker-001812F-est_gff_StringTie-gene-0.0-mRNA-1
23194,his1.3_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LASC022853,"hypothetical protein,Histone H1.3",NaN,248,744,maker-001826F-exonerate_protein2genome-gene-0.2,maker-001826F-exonerate_protein2genome-gene-0....


In [12]:
combined_anno["silkGroup"].value_counts()

silkGroup
Spidroin     13
Spice-LMa     6
Name: count, dtype: int64

In [13]:
merged_combined[merged_combined["cluster"]== "ZoneA_MaSp1"]

,geneName,cluster,avg_log2FC,p_val_adj,pct.1,pct.2,rank,geneNameProtein,silkGroup,lascID,product,Group,aa_len,tx_len,gene_id,tx_name,Lsc.v1.1.genes.AA.signalP.lascID,Orthogroup
152,LASC001841,ZoneA_MaSp1,0.679838,3.245226e-06,0.419,0.248,895.0,NaN,NaN,LASC001841,"hypothetical, protein",NaN,85,255,augustus_masked-000010F-processed-gene-33.12,augustus_masked-000010F-processed-gene-33.12-m...,LASC001841,OG0143392
155,eprs,ZoneA_MaSp1,1.583229,1.399239e-55,0.851,0.338,58.0,NaN,NaN,LASC001860,"Bifunctional, glutamate/proline--tRNA, ligase",NaN,1610,4830,augustus_masked-000010F-processed-gene-40.5,augustus_masked-000010F-processed-gene-40.5-mR...,LASC001860,OG0001914
199,nf2,ZoneA_MaSp1,1.068441,6.606550e-22,0.645,0.286,541.0,NaN,NaN,LASC002383,Merlin,NaN,599,1797,augustus_masked-000014F-processed-gene-48.5,augustus_masked-000014F-processed-gene-48.5-mR...,LASC002383,OG0001266
340,plrg1,ZoneA_MaSp1,0.970954,4.544236e-05,0.537,0.337,925.0,NaN,NaN,LASC004322,"Pleiotropic, regulator, 1",NaN,474,1422,augustus_masked-000034F-processed-gene-0.7,augustus_masked-000034F-processed-gene-0.7-mRNA-1,LASC004322,OG0005893
351,zc3h14,ZoneA_MaSp1,0.400350,9.928516e-32,0.627,0.252,337.0,NaN,NaN,LASC004405,"Zinc, finger, CCCH, domain-containing, protein...",NaN,538,1614,augustus_masked-000034F-processed-gene-39.2,augustus_masked-000034F-processed-gene-39.2-mR...,LASC004405,OG0004973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22680,LASC022175,ZoneA_MaSp1,1.239089,2.465971e-32,0.494,0.223,326.0,NaN,NaN,LASC022175,hypothetical protein,NaN,420,5166,maker-000609F-est_gff_StringTie-gene-0.1,maker-000609F-est_gff_StringTie-gene-0.1-mRNA-2,LASC022175,OG0007274
22704,ttll1,ZoneA_MaSp1,0.957683,1.862083e-07,0.483,0.249,869.0,NaN,NaN,LASC022212,Probable tubulin polyglutamylase TTLL1,NaN,380,7938,maker-000619F-augustus-gene-0.4,maker-000619F-augustus-gene-0.4-mRNA-1,LASC022212,OG0001333
22707,LASC022211,ZoneA_MaSp1,0.775109,3.618151e-11,0.545,0.301,794.0,NaN,NaN,LASC022211,hypothetical protein,NaN,29,5186,maker-000619F-est_gff_StringTie-gene-0.7,maker-000619F-est_gff_StringTie-gene-0.7-mRNA-1,LASC022211,OG0012717
22851,cbx8,ZoneA_MaSp1,1.118895,1.268930e-08,0.457,0.251,848.0,NaN,NaN,LASC022391,Chromobox protein homolog 8,NaN,435,9125,maker-000673F-est_gff_StringTie-gene-0.128,maker-000673F-est_gff_StringTie-gene-0.128-mRNA-1,LASC022391,OG0001504


In [14]:
merged_combined.to_csv("C:/Users/46705/Documents/SpiderSilk/data/annotations/annotatoins_LS.csv")

In [15]:
df_transposed = df_anno.T
df_transposed = df_transposed.iloc[:, :3]
df_transposed
df_transposed.columns = df_transposed.iloc[0]
df_transposed = df_transposed.drop(df_transposed.index[0])
df_transposed = df_transposed.iloc[6:, :]
df_transposed.to_csv("C:/Users/46705/Documents/SpiderSilk/data/annotations/spidroin_OG.csv")